In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

## 1. Load Results

In [ ]:
# Load PCA results
df = pd.read_csv('eval/PCA_pipeline/metrics/PCA_all_results.csv')

print(f"Total datasets: {len(df)}")
print(f"Columns: {df.columns.tolist()}")
df.head()

## 2. Overall Performance Statistics

In [ ]:
# Key metrics
key_metrics = ['VUS-PR', 'VUS-ROC', 'Affiliation_Precision', 'Affiliation_Recall']

print("="*80)
print("PCA Performance Summary")
print("="*80)

for metric in key_metrics:
    if metric in df.columns:
        mean_val = df[metric].mean()
        std_val = df[metric].std()
        median_val = df[metric].median()
        min_val = df[metric].min()
        max_val = df[metric].max()
        
        print(f"\n{metric}:")
        print(f"  Mean:   {mean_val:.4f} ± {std_val:.4f}")
        print(f"  Median: {median_val:.4f}")
        print(f"  Range:  [{min_val:.4f}, {max_val:.4f}]")

# Runtime statistics
print(f"\nRuntime Statistics:")
print(f"  Mean:   {df['time'].mean():.3f}s ± {df['time'].std():.3f}s")
print(f"  Median: {df['time'].median():.3f}s")
print(f"  Total:  {df['time'].sum():.2f}s ({df['time'].sum()/60:.2f} minutes)")
print("="*80)

## 3. Distribution Analysis

In [ ]:
# Distribution plots for key metrics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for idx, metric in enumerate(key_metrics):
    if metric in df.columns:
        ax = axes[idx]
        
        # Histogram with KDE
        ax.hist(df[metric], bins=30, alpha=0.6, color='skyblue', edgecolor='black', density=True)
        
        # KDE overlay
        from scipy.stats import gaussian_kde
        kde = gaussian_kde(df[metric].dropna())
        x_range = np.linspace(df[metric].min(), df[metric].max(), 100)
        ax.plot(x_range, kde(x_range), 'r-', linewidth=2, label='KDE')
        
        # Mean and median lines
        ax.axvline(df[metric].mean(), color='green', linestyle='--', linewidth=2, label=f'Mean: {df[metric].mean():.3f}')
        ax.axvline(df[metric].median(), color='orange', linestyle='--', linewidth=2, label=f'Median: {df[metric].median():.3f}')
        
        ax.set_xlabel(metric, fontweight='bold')
        ax.set_ylabel('Density', fontweight='bold')
        ax.set_title(f'Distribution of {metric}', fontweight='bold', fontsize=13)
        ax.legend()
        ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('eval/PCA_pipeline/metrics/distribution_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Distribution plot saved to: eval/PCA_pipeline/metrics/distribution_analysis.png")

## 4. Box Plot Analysis

In [ ]:
# Box plots for all key metrics
fig, ax = plt.subplots(figsize=(12, 6))

available_metrics = [m for m in key_metrics if m in df.columns]
data_to_plot = [df[m].dropna() for m in available_metrics]

bp = ax.boxplot(data_to_plot, labels=available_metrics, patch_artist=True,
                showmeans=True, meanline=True,
                meanprops=dict(color='red', linestyle='--', linewidth=2))

# Color the boxes
colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow']
for patch, color in zip(bp['boxes'], colors[:len(bp['boxes'])]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.set_ylabel('Score', fontweight='bold', fontsize=12)
ax.set_title('PCA Performance Metrics - Box Plot Analysis', fontweight='bold', fontsize=14)
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=15, ha='right')

plt.tight_layout()
plt.savefig('eval/PCA_pipeline/metrics/boxplot_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Box plot saved to: eval/PCA_pipeline/metrics/boxplot_analysis.png")

## 5. Performance by Dataset Category

Analyze performance across different data sources (NAB, WSD, MSL, Stock, etc.)

In [ ]:
# Extract dataset category from filename
def extract_category(filename):
    parts = filename.split('_')
    if len(parts) > 1:
        return parts[1]  # NAB, WSD, MSL, Stock, etc.
    return 'Unknown'

df['category'] = df['file'].apply(extract_category)

# Count datasets per category
category_counts = df['category'].value_counts()
print("\nDatasets per category:")
print(category_counts.head(15))

# Performance by category (for categories with >5 datasets)
min_samples = 5
valid_categories = category_counts[category_counts >= min_samples].index

print(f"\nAnalyzing categories with ≥{min_samples} datasets: {len(valid_categories)} categories")

In [ ]:
# VUS-PR by category
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Filter data
df_filtered = df[df['category'].isin(valid_categories)]

# Box plot
ax1 = axes[0]
category_order = df_filtered.groupby('category')['VUS-PR'].median().sort_values(ascending=False).index
sns.boxplot(data=df_filtered, x='category', y='VUS-PR', order=category_order, ax=ax1)
ax1.set_xlabel('Data Category', fontweight='bold', fontsize=12)
ax1.set_ylabel('VUS-PR', fontweight='bold', fontsize=12)
ax1.set_title('PCA Performance by Dataset Category', fontweight='bold', fontsize=13)
ax1.tick_params(axis='x', rotation=45)
ax1.grid(axis='y', alpha=0.3)

# Mean comparison
ax2 = axes[1]
category_means = df_filtered.groupby('category')['VUS-PR'].agg(['mean', 'std', 'count'])
category_means = category_means.sort_values('mean', ascending=False)

x_pos = np.arange(len(category_means))
ax2.bar(x_pos, category_means['mean'], yerr=category_means['std'], 
        alpha=0.7, capsize=5, color='steelblue', edgecolor='black')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(category_means.index, rotation=45, ha='right')
ax2.set_xlabel('Data Category', fontweight='bold', fontsize=12)
ax2.set_ylabel('Mean VUS-PR', fontweight='bold', fontsize=12)
ax2.set_title('Mean VUS-PR by Category (with std)', fontweight='bold', fontsize=13)
ax2.grid(axis='y', alpha=0.3)

# Add sample sizes as text
for i, (idx, row) in enumerate(category_means.iterrows()):
    ax2.text(i, row['mean'] + row['std'] + 0.02, f"n={int(row['count'])}", 
            ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('eval/PCA_pipeline/metrics/performance_by_category.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "="*60)
print("Best Performing Categories (by mean VUS-PR):")
print("="*60)
print(category_means.head(10).to_string())
print("\n✓ Category analysis plot saved to: eval/PCA_pipeline/metrics/performance_by_category.png")

## 6. Correlation Analysis

Examine relationships between dataset characteristics and performance

In [ ]:
# Correlation matrix
correlation_cols = ['data_length', 'num_anomalies', 'anomaly_ratio', 'sliding_window', 
                    'VUS-PR', 'VUS-ROC', 'time']
corr_data = df[correlation_cols].corr()

# Heatmap
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_data, annot=True, fmt='.3f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Correlation Matrix: Dataset Characteristics vs Performance', 
             fontweight='bold', fontsize=13, pad=20)
plt.tight_layout()
plt.savefig('eval/PCA_pipeline/metrics/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Correlation matrix saved to: eval/PCA_pipeline/metrics/correlation_matrix.png")

In [ ]:
# Scatter plots: key relationships
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. VUS-PR vs Data Length
axes[0, 0].scatter(df['data_length'], df['VUS-PR'], alpha=0.5, s=50)
axes[0, 0].set_xlabel('Data Length', fontweight='bold')
axes[0, 0].set_ylabel('VUS-PR', fontweight='bold')
axes[0, 0].set_title('VUS-PR vs Data Length', fontweight='bold')
axes[0, 0].grid(alpha=0.3)
# Add trend line
z = np.polyfit(df['data_length'], df['VUS-PR'], 1)
p = np.poly1d(z)
axes[0, 0].plot(df['data_length'], p(df['data_length']), "r--", alpha=0.8, linewidth=2)

# 2. VUS-PR vs Anomaly Ratio
axes[0, 1].scatter(df['anomaly_ratio']*100, df['VUS-PR'], alpha=0.5, s=50, c='green')
axes[0, 1].set_xlabel('Anomaly Ratio (%)', fontweight='bold')
axes[0, 1].set_ylabel('VUS-PR', fontweight='bold')
axes[0, 1].set_title('VUS-PR vs Anomaly Ratio', fontweight='bold')
axes[0, 1].grid(alpha=0.3)
z = np.polyfit(df['anomaly_ratio'], df['VUS-PR'], 1)
p = np.poly1d(z)
axes[0, 1].plot(df['anomaly_ratio']*100, p(df['anomaly_ratio']), "r--", alpha=0.8, linewidth=2)

# 3. Runtime vs Data Length
axes[1, 0].scatter(df['data_length'], df['time'], alpha=0.5, s=50, c='orange')
axes[1, 0].set_xlabel('Data Length', fontweight='bold')
axes[1, 0].set_ylabel('Runtime (seconds)', fontweight='bold')
axes[1, 0].set_title('Runtime vs Data Length', fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# 4. VUS-PR vs VUS-ROC
axes[1, 1].scatter(df['VUS-ROC'], df['VUS-PR'], alpha=0.5, s=50, c='purple')
axes[1, 1].set_xlabel('VUS-ROC', fontweight='bold')
axes[1, 1].set_ylabel('VUS-PR', fontweight='bold')
axes[1, 1].set_title('VUS-PR vs VUS-ROC', fontweight='bold')
axes[1, 1].grid(alpha=0.3)
# Add diagonal line
lims = [0, 1]
axes[1, 1].plot(lims, lims, 'k--', alpha=0.5, linewidth=1)

plt.tight_layout()
plt.savefig('eval/PCA_pipeline/metrics/scatter_relationships.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Scatter plot analysis saved to: eval/PCA_pipeline/metrics/scatter_relationships.png")

## 7. Top and Bottom Performers

In [ ]:
# Top 10 datasets by VUS-PR
top_10 = df.nlargest(10, 'VUS-PR')[['file', 'VUS-PR', 'VUS-ROC', 'category', 'anomaly_ratio', 'time']]
print("="*80)
print("Top 10 Datasets by VUS-PR:")
print("="*80)
print(top_10.to_string(index=False))

# Bottom 10 datasets by VUS-PR
bottom_10 = df.nsmallest(10, 'VUS-PR')[['file', 'VUS-PR', 'VUS-ROC', 'category', 'anomaly_ratio', 'time']]
print("\n" + "="*80)
print("Bottom 10 Datasets by VUS-PR:")
print("="*80)
print(bottom_10.to_string(index=False))

In [ ]:
# Visualize top and bottom performers
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Top 10
top_10_sorted = top_10.sort_values('VUS-PR', ascending=True)
y_pos = np.arange(len(top_10_sorted))
ax1.barh(y_pos, top_10_sorted['VUS-PR'], color='green', alpha=0.7, edgecolor='black')
ax1.set_yticks(y_pos)
ax1.set_yticklabels([f.split('_')[0:3] for f in top_10_sorted['file']], fontsize=9)
ax1.set_xlabel('VUS-PR', fontweight='bold', fontsize=12)
ax1.set_title('Top 10 Datasets', fontweight='bold', fontsize=13, color='green')
ax1.grid(axis='x', alpha=0.3)
for i, v in enumerate(top_10_sorted['VUS-PR']):
    ax1.text(v + 0.01, i, f'{v:.4f}', va='center', fontsize=9, fontweight='bold')

# Bottom 10
bottom_10_sorted = bottom_10.sort_values('VUS-PR', ascending=False)
y_pos = np.arange(len(bottom_10_sorted))
ax2.barh(y_pos, bottom_10_sorted['VUS-PR'], color='red', alpha=0.7, edgecolor='black')
ax2.set_yticks(y_pos)
ax2.set_yticklabels([f.split('_')[0:3] for f in bottom_10_sorted['file']], fontsize=9)
ax2.set_xlabel('VUS-PR', fontweight='bold', fontsize=12)
ax2.set_title('Bottom 10 Datasets', fontweight='bold', fontsize=13, color='red')
ax2.grid(axis='x', alpha=0.3)
for i, v in enumerate(bottom_10_sorted['VUS-PR']):
    ax2.text(v + 0.01, i, f'{v:.4f}', va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('eval/PCA_pipeline/metrics/top_bottom_performers.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Top/Bottom performers plot saved to: eval/PCA_pipeline/metrics/top_bottom_performers.png")

## 8. Statistical Tests

Normality tests and statistical comparisons

In [ ]:
# Normality tests
print("="*80)
print("Normality Tests (Shapiro-Wilk)")
print("="*80)

for metric in key_metrics:
    if metric in df.columns:
        data = df[metric].dropna()
        stat, p_value = stats.shapiro(data)
        is_normal = "Yes" if p_value > 0.05 else "No"
        print(f"{metric}:")
        print(f"  Statistic: {stat:.4f}")
        print(f"  p-value: {p_value:.4f}")
        print(f"  Normal distribution (α=0.05): {is_normal}\n")

In [ ]:
# Q-Q plots for normality visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for idx, metric in enumerate(key_metrics):
    if metric in df.columns:
        stats.probplot(df[metric].dropna(), dist="norm", plot=axes[idx])
        axes[idx].set_title(f'Q-Q Plot: {metric}', fontweight='bold', fontsize=12)
        axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('eval/PCA_pipeline/metrics/qq_plots.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Q-Q plots saved to: eval/PCA_pipeline/metrics/qq_plots.png")

## 9. Performance Quartiles Analysis

In [ ]:
# Divide datasets into quartiles based on VUS-PR
df['VUS_PR_quartile'] = pd.qcut(df['VUS-PR'], q=4, labels=['Q1 (Lowest)', 'Q2', 'Q3', 'Q4 (Highest)'])

print("="*80)
print("Performance Quartiles (based on VUS-PR)")
print("="*80)

for quartile in ['Q1 (Lowest)', 'Q2', 'Q3', 'Q4 (Highest)']:
    q_data = df[df['VUS_PR_quartile'] == quartile]
    print(f"\n{quartile}:")
    print(f"  Count: {len(q_data)}")
    print(f"  VUS-PR range: [{q_data['VUS-PR'].min():.4f}, {q_data['VUS-PR'].max():.4f}]")
    print(f"  Mean VUS-PR: {q_data['VUS-PR'].mean():.4f}")
    print(f"  Mean anomaly ratio: {q_data['anomaly_ratio'].mean()*100:.2f}%")
    print(f"  Mean data length: {q_data['data_length'].mean():.0f}")
    print(f"  Top categories: {q_data['category'].value_counts().head(3).to_dict()}")

## 10. Export Summary Report

In [ ]:
# Create a comprehensive summary report
summary_report = {
    'Total Datasets': len(df),
    'Mean VUS-PR': df['VUS-PR'].mean(),
    'Std VUS-PR': df['VUS-PR'].std(),
    'Median VUS-PR': df['VUS-PR'].median(),
    'Mean VUS-ROC': df['VUS-ROC'].mean(),
    'Std VUS-ROC': df['VUS-ROC'].std(),
    'Mean Runtime (s)': df['time'].mean(),
    'Total Runtime (min)': df['time'].sum() / 60,
    'Best Dataset': df.loc[df['VUS-PR'].idxmax(), 'file'],
    'Best VUS-PR': df['VUS-PR'].max(),
    'Worst Dataset': df.loc[df['VUS-PR'].idxmin(), 'file'],
    'Worst VUS-PR': df['VUS-PR'].min(),
    'Unique Categories': df['category'].nunique(),
}

summary_df = pd.DataFrame([summary_report]).T
summary_df.columns = ['Value']
summary_df.to_csv('eval/PCA_pipeline/metrics/PCA_summary_report.csv')

print("="*80)
print("SUMMARY REPORT")
print("="*80)
print(summary_df.to_string())
print("\n✓ Summary report saved to: eval/PCA_pipeline/metrics/PCA_summary_report.csv")

## 11. Final Comprehensive Visualization

In [ ]:
# Create a comprehensive dashboard
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. VUS-PR Distribution
ax1 = fig.add_subplot(gs[0, 0])
ax1.hist(df['VUS-PR'], bins=30, alpha=0.7, color='steelblue', edgecolor='black')
ax1.axvline(df['VUS-PR'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
ax1.set_xlabel('VUS-PR', fontweight='bold')
ax1.set_ylabel('Frequency', fontweight='bold')
ax1.set_title('VUS-PR Distribution', fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# 2. VUS-ROC Distribution
ax2 = fig.add_subplot(gs[0, 1])
ax2.hist(df['VUS-ROC'], bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
ax2.axvline(df['VUS-ROC'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
ax2.set_xlabel('VUS-ROC', fontweight='bold')
ax2.set_ylabel('Frequency', fontweight='bold')
ax2.set_title('VUS-ROC Distribution', fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

# 3. Runtime Distribution
ax3 = fig.add_subplot(gs[0, 2])
ax3.hist(df['time'], bins=30, alpha=0.7, color='coral', edgecolor='black')
ax3.axvline(df['time'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
ax3.set_xlabel('Runtime (s)', fontweight='bold')
ax3.set_ylabel('Frequency', fontweight='bold')
ax3.set_title('Runtime Distribution', fontweight='bold')
ax3.legend()
ax3.grid(alpha=0.3)

# 4. Category Performance
ax4 = fig.add_subplot(gs[1, :])
if len(valid_categories) > 0:
    df_cat = df[df['category'].isin(valid_categories)]
    category_stats = df_cat.groupby('category')['VUS-PR'].mean().sort_values(ascending=False).head(15)
    category_stats.plot(kind='bar', ax=ax4, color='teal', alpha=0.7, edgecolor='black')
    ax4.set_xlabel('Category', fontweight='bold')
    ax4.set_ylabel('Mean VUS-PR', fontweight='bold')
    ax4.set_title('Top 15 Categories by Mean VUS-PR', fontweight='bold')
    ax4.tick_params(axis='x', rotation=45)
    ax4.grid(axis='y', alpha=0.3)

# 5. VUS-PR vs Anomaly Ratio
ax5 = fig.add_subplot(gs[2, 0])
scatter = ax5.scatter(df['anomaly_ratio']*100, df['VUS-PR'], 
                     c=df['data_length'], s=30, alpha=0.6, cmap='viridis')
ax5.set_xlabel('Anomaly Ratio (%)', fontweight='bold')
ax5.set_ylabel('VUS-PR', fontweight='bold')
ax5.set_title('VUS-PR vs Anomaly Ratio', fontweight='bold')
ax5.grid(alpha=0.3)
plt.colorbar(scatter, ax=ax5, label='Data Length')

# 6. VUS-PR vs Data Length
ax6 = fig.add_subplot(gs[2, 1])
ax6.scatter(df['data_length'], df['VUS-PR'], alpha=0.5, s=30, color='purple')
ax6.set_xlabel('Data Length', fontweight='bold')
ax6.set_ylabel('VUS-PR', fontweight='bold')
ax6.set_title('VUS-PR vs Data Length', fontweight='bold')
ax6.grid(alpha=0.3)

# 7. VUS-PR vs VUS-ROC
ax7 = fig.add_subplot(gs[2, 2])
ax7.scatter(df['VUS-ROC'], df['VUS-PR'], alpha=0.5, s=30, color='orange')
ax7.plot([0, 1], [0, 1], 'k--', alpha=0.3)
ax7.set_xlabel('VUS-ROC', fontweight='bold')
ax7.set_ylabel('VUS-PR', fontweight='bold')
ax7.set_title('VUS-PR vs VUS-ROC', fontweight='bold')
ax7.grid(alpha=0.3)

fig.suptitle('PCA Performance Dashboard - TSB-AD-U', fontsize=16, fontweight='bold', y=0.995)

plt.savefig('eval/PCA_pipeline/metrics/comprehensive_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Comprehensive dashboard saved to: eval/PCA_pipeline/metrics/comprehensive_dashboard.png")

## 12. Conclusions

Summary of key findings from the PCA analysis.

In [ ]:
print("="*80)
print("KEY FINDINGS - PCA ANOMALY DETECTION ON TSB-AD-U")
print("="*80)

print(f"\n1. OVERALL PERFORMANCE:")
print(f"   • Evaluated on {len(df)} datasets")
print(f"   • Mean VUS-PR: {df['VUS-PR'].mean():.4f} (±{df['VUS-PR'].std():.4f})")
print(f"   • Mean VUS-ROC: {df['VUS-ROC'].mean():.4f} (±{df['VUS-ROC'].std():.4f})")

print(f"\n2. PERFORMANCE RANGE:")
print(f"   • Best VUS-PR: {df['VUS-PR'].max():.4f} ({df.loc[df['VUS-PR'].idxmax(), 'file']})")
print(f"   • Worst VUS-PR: {df['VUS-PR'].min():.4f} ({df.loc[df['VUS-PR'].idxmin(), 'file']})")
print(f"   • Performance spread: {df['VUS-PR'].max() - df['VUS-PR'].min():.4f}")

print(f"\n3. COMPUTATIONAL EFFICIENCY:")
print(f"   • Mean runtime: {df['time'].mean():.3f}s per dataset")
print(f"   • Total runtime: {df['time'].sum()/60:.2f} minutes for all datasets")
print(f"   • Runtime/datapoint: {(df['time'].sum() / df['data_length'].sum())*1000:.4f} ms/point")

print(f"\n4. CATEGORY INSIGHTS:")
if len(valid_categories) > 0:
    best_cat = df[df['category'].isin(valid_categories)].groupby('category')['VUS-PR'].mean().idxmax()
    best_cat_score = df[df['category'].isin(valid_categories)].groupby('category')['VUS-PR'].mean().max()
    print(f"   • Best performing category: {best_cat} (mean VUS-PR: {best_cat_score:.4f})")
    print(f"   • Analyzed {len(valid_categories)} categories with ≥{min_samples} datasets")

print(f"\n5. KEY CORRELATIONS:")
corr_ar = df[['anomaly_ratio', 'VUS-PR']].corr().iloc[0, 1]
corr_len = df[['data_length', 'VUS-PR']].corr().iloc[0, 1]
print(f"   • VUS-PR vs Anomaly Ratio: r={corr_ar:.3f}")
print(f"   • VUS-PR vs Data Length: r={corr_len:.3f}")

print("\n" + "="*80)
print("Analysis complete! All plots and reports saved to: eval/PCA_pipeline/metrics/")
print("="*80)